In [1]:
using Distributed

In [2]:
procs = addprocs(6)

6-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7

In [3]:
length(procs)

6

In [4]:
@everywhere begin

const d = 4
const N = 41
const P = Int(floor(2*N/3))
    
const nbits = max(53, Int(floor(P*log2(10))))
setprecision(BigFloat, nbits)

const c_text = "0.5754" #0.575303744708453
const r_text = "0.8"
    
const c = BigFloat(c_text)
const r = BigFloat(r_text)

prefix = "feig_d$(d)_N$(N)_P$(P)_c$(c_text)_r$(r_text)" 
    
end

mkpath(prefix)

d, N, P, nbits, c, r

(4, 41, 27, 89, 0.5753999999999999999999999996, 0.8000000000000000000000000006)

In [5]:
@everywhere begin

import Base: <, <=, ==, >, >=
import Base: zero, one
import Base: abs
import Base: +, -, *, /, ^

struct Poly
    coeffs::Array{BigFloat, 1}
    cr::Tuple{BigFloat, BigFloat}
    Poly(a::Array{BigFloat, 1}, cr) = length(a) == 1+N ? new(a, cr) : error("wrong #coeffs")
end

abs(t::Poly) = sum(abs(a) for a in t.coeffs)
zero(p::Poly) = Poly(fill(BigFloat(0), N+1), p.cr)
one(p::Poly) = Poly([k==0 ? BigFloat(1) : BigFloat(0) for k in 0:N], p.cr)

(-)(p::Poly) = Poly(-p.coeffs, p.cr)

(*)(p::Poly, a::Number) = Poly(p.coeffs*a, p.cr)
(*)(a::Number, p::Poly) = p*a

(/)(p::Poly, a::BigFloat) = p*(1/a)

function +(p::Poly, a::BigFloat)
    coeffs = copy(p.coeffs)
    coeffs[1] += a
    Poly(coeffs, p.cr)
end

(+)(a::BigFloat, p::Poly) = p+a
(-)(p::Poly, a::BigFloat) = p+(-a)

(+)(p::Poly, q::Poly) = p.cr == q.cr ? Poly(p.coeffs+q.coeffs, p.cr) : error("!")
(-)(p::Poly, q::Poly) = p.cr == q.cr ? Poly(p.coeffs-q.coeffs, p.cr) : error("!")

(*)(a::Array{BigFloat, 1}, b::BigFloat) = [ak*b for ak in a]
(*)(b::BigFloat, a::Array{BigFloat, 1}) = a*b

(/)(a::Array{BigFloat, 1}, b::BigFloat) = [ak/b for ak in a]

function *(p::Poly, q::Poly)
    if p.cr != q.cr error("!") end
    coeffs = fill(BigFloat(0), N+1)
    for j in 0:N
        for k in 0:N
            if j+k <= N
                coeffs[1+j+k] += p.coeffs[1+j]*q.coeffs[1+k]
            end
        end
    end
    Poly(coeffs, p.cr)
end

basis_element(p::Poly, k::Integer) = Poly([j==k ? BigFloat(1) : BigFloat(0) for j in 0:N], p.cr)

function identity(p::Poly)
    c, r = p.cr
    coeffs = fill(BigFloat(0), N+1)
    coeffs[1] = c
    coeffs[2] = r
    Poly(coeffs, p.cr)
end

function diff(p::Poly)
    c, r = p.cr
    Poly([k == N+1 ? BigFloat(0) : k*p.coeffs[1+k] for k in 1:N+1], p.cr)/r
end

# Power
function russian(t, p::Integer)
    if p == 0
        return one(t)
    elseif p == 1
        return t
    end
    
    # ignore trailing zeros in binary expansion
    trailing = trailing_zeros(p) + 1
    p >>= trailing
    
    # repeatedly square
    while (trailing -= 1) > 0
        t *= t
    end
    
    # compute the rest
    u = t
    while p > 0
        trailing = trailing_zeros(p) + 1
        p >>= trailing
        while (trailing -= 1) >= 0
            t *= t
        end
        u *= t
    end
    u
end

(^)(t::Poly, p::Integer) = russian(t, p)

# Evaluation and composition of polynomial.
function (t::Poly)(x::Union{BigFloat, Poly, Complex{BigFloat}})
    c, r = t.cr
    u = (x-c)/r
    total = t.coeffs[N+1]*u + t.coeffs[N]
    for k in N-1:-1:1
        total = total*u + t.coeffs[k]
    end
    total
end

(t::Poly)(x::Number) = t(BigFloat(x))
    
end

In [6]:
@everywhere begin

coeffs = fill(BigFloat(0), N+1)
mu_infty = BigFloat("-1.5949013562288205644978287")
coeffs[1:2] = [BigFloat(1), mu_infty]
p0 = Poly(coeffs, (BigFloat(0), BigFloat(1)))

zero_Omega = zero(Poly(fill(BigFloat(0), N+1), (c, r)))
I_Omega = identity(zero_Omega)

g0 = p0(I_Omega)
    
end

In [7]:
@everywhere poly_basis = [basis_element(g0, k) for k in 0:N];

In [8]:
@everywhere begin

Q(z) = z^d
Qdash(z) = d == 2 ? 2*z : d*z^(d-1)

function T(f)
    x = identity(f)
    a = f(1)
    (1/a)*f(Q(f(x*Q(a))))
end

function T(g)
    x = identity(g)
    
    a = g(1)
    Qa = Q(a)
    xQa = x*Qa
    gxQa = g(xQa)
    QgxQa = Q(gxQa)
    gQgxQa = g(QgxQa)
    Tg = (1/a)*gQgxQa
end

function T_and_DT(g)
    x = identity(g)
    
    a = g(1)
    Qa = Q(a)
    xQa = x*Qa
    gxQa = g(xQa)
    QgxQa = Q(gxQa)
    gQgxQa = g(QgxQa)
    Tg = (1/a)*gQgxQa
    
    g_dash = diff(g)
    g_dashxQa = g_dash(xQa)
    g_dashQgxQa = g_dash(QgxQa)
    
    dgxQadm1 = d*gxQa^(d-1)
    
    # we should ensure that any subexpressions
    # that do not depend on delta_g
    # are computed in the outer function, here.
    
    function DTg(delta_g)
        delta_a = delta_g(1)
        delta_Qa = d*a^(d-1)*delta_a
        delta_xQa = x*delta_Qa
        delta_gxQa = delta_g(xQa) + g_dashxQa*delta_xQa
        delta_QgxQa = dgxQadm1*delta_gxQa
        delta_gQgxQa = delta_g(QgxQa) + g_dashQgxQa*delta_QgxQa
        delta_Tg = (-1/a^2)*delta_a*gQgxQa + (1/a)*delta_gQgxQa
    end
    
    Tg, DTg
end
    
end

In [9]:
@everywhere begin

function eye(T::Type, n::Integer)
    I::Array{T, 2} = [j==k ? T(1) : T(0) for k in 1:n, j in 1:n]
end

# be careful that matrix_elements is only used from 1 process.
# pmap can then be used below.
# for large degree, this should help significantly.
# when the function being applied is a closure,
# there might be a benefit to using CachingPool(workers())
# as an additional, second, argument to pmap.
# actually, here, DTp is not a closure...
    
function matrix_elements(L, basis)
    J_polys = pmap(L, basis)
    #J_polys = pmap(L, CachingPool(workers()), basis)
    J = [J_polys[k].coeffs[j] for j in 1:N+1, k in 1:N+1]
end
    
function newtonstep(p::Poly)
    Tp, DTp = T_and_DT(p)
    Fp = Tp - p
    J = matrix_elements(DTp, poly_basis)
    I = eye(BigFloat, N+1)
    q = Poly(p.coeffs - inv(J-I)*Fp.coeffs, p.cr)
end
    
function newton(p0::Poly, n::Integer)
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1 = newtonstep(p0)
        error = abs(T(p1)-p1)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end
    
end

## Skip if precomputed

In [10]:
ga = g0
gb = T(ga)
erra = abs(gb-ga)
for k in 1:20
    gc = T(gb)
    errb = abs(gc - gb)
    println("$(k) $(errb)")
    if errb < erra
        gb, ga = gc, gb
        erra = errb
    else
        break
    end
end

1 0.300601521664247044652198378
2 0.02346509242791335839763515382
3 0.0180426574284850493953795104
4 0.001900115087433402971279775082
5 0.001275689040555407408813294032
6 0.0001662216219179982646270172341
7 9.805192134014271184838466372e-05
8 1.585895595558708391845038489e-05
9 7.819496162817914749060600082e-06
10 1.477055958776234932531946545e-06
11 6.3516617872692938199616042e-07
12 1.334475019895346782541765221e-07
13 5.217770131948772800612501723e-08
14 1.185084187294380968232706253e-08
15 4.321500646820411318768239282e-09
16 1.045085488583884904204801862e-09
17 4.55397689184676522618084147e-10
18 2.080971293226493440240470112e-09


In [11]:
using Serialization

In [11]:
open(f -> serialize(f, ga), "$(prefix)/ga.jls", "w")

In [12]:
ga = open(deserialize, "$(prefix)/ga.jls")

Poly(BigFloat[-0.0001511713586285057764382956007, -1.256390384719997063102552342, 0.2468156476186510133959630149, 0.05671913432480105149197335683, -0.03593899502915159445308593625, 0.004309827920932066272417000396, 0.001963540983829639871628245721, -0.0008943234096129327486414316774, 6.105399126087436803424560035e-05, 6.188331889293793023558802634e-05  …  -8.193351300176035663556334101e-17, 4.678653983445518422126053491e-18, 6.290975100221391234675622028e-18, -2.03660205890966901812374007e-18, -5.51448882292050912519337327e-20, 2.007811849831702745068772803e-19, -4.634037823704109267536956511e-20, -6.570702792452528126718463147e-21, 5.946188525464014539130369074e-21, -9.218629861643623996582854476e-22], (0.5753999999999999999999999996, 0.8000000000000000000000000006))

In [13]:
ga.coeffs[1]

-0.0001511713586285057764382956007

In [14]:
abs(T(ga)-ga)

4.55397689184676522618084147e-10

In [15]:
#precompile
newton(g0, 0);

In [16]:
@time gn = newton(ga, 20);

1: 5.771641951697389541109493618e-20
2: 2.902662907434221195203508284e-26
3: 2.107737285904556334282003902e-26
  2.165020 seconds (10.72 M allocations: 507.546 MiB, 6.15% gc time)


In [17]:
using Serialization
open(f -> serialize(f, gn), "$(prefix)/g.jls", "w")

In [17]:
@everywhere begin
    
using Serialization

gn = open(deserialize, "$(prefix)/g.jls")
    
end

In [18]:
gn(0)

0.9999999999999999999999999984

In [19]:
gn(1)

-0.5916099166344381501396243519

In [20]:
1/gn(1)

-1.690302971405244853343780156

In [21]:
residue_approx = T(gn)-gn
abs(residue_approx)

2.107737285904556334282003902e-26

In [22]:
ccc = gn.coeffs[1]

-0.0001511713350611291656840259461

In [23]:
a = gn(1)
x = identity(gn)
xQa = x*Q(a)
psi(x) = (x-c)/r
norm1 = abs(psi(xQa))

0.7536423257237438864255931463

In [24]:
QgxQa = Q(gn(xQa))
norm2 = abs(psi(QgxQa))

0.8034565068330793155746543136

In [25]:
if norm1 >= 1 || norm2 >= 1
    println("Proof hopeless with these domains.")
end

In [26]:
Tgn, DTgn = T_and_DT(gn)

(Poly(BigFloat[-0.0001511713350611291656840145904, -1.256390384839309113454966122, 0.2468156475594798891903756126, 0.05671913449535499518131215488, -0.03593899510003976630438281722, 0.004309827907215549990447627635, 0.001963541004817085940087265941, -0.0008943234149703396950331270934, 6.105398965710550909211669004e-05, 6.18833202716769579476806784e-05  …  -8.193352719716318716845210401e-17, 4.678651669174662819840084076e-18, 6.290977208505213125174331796e-18, -2.036602398263703289116784981e-18, -5.514501237092254757475084934e-20, 2.007812537370097757991522041e-19, -4.634038429218728645259136977e-20, -6.570708157395391346804408734e-21, 5.946190599978746106321751204e-21, -9.218630047818848213294095374e-22], (0.5753999999999999999999999996, 0.8000000000000000000000000006)), var"#DTg#15"{Poly,BigFloat,Poly,Poly,Poly,Poly,Poly,Poly}(Poly(BigFloat[0.5753999999999999999999999996, 0.8000000000000000000000000006, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [27]:
@time J = matrix_elements(DTgn, poly_basis);

  0.084212 seconds (106.17 k allocations: 3.575 MiB)


In [28]:
using Serialization
open(f -> serialize(f, gn), "$(prefix)/g.jls", "w")
open(f -> serialize(f, J), "$(prefix)/jac.jls", "w")

## Continue...

In [29]:
@everywhere begin
    
using Serialization

gn = open(deserialize, "$(prefix)/g.jls")
J = open(deserialize, "$(prefix)/jac.jls")
    
end

In [30]:
gn.cr == (c, r)

true

In [31]:
Tgn, DTgn = T_and_DT(gn) # only if not done above

(Poly(BigFloat[-0.0001511713350611291656840145904, -1.256390384839309113454966122, 0.2468156475594798891903756126, 0.05671913449535499518131215488, -0.03593899510003976630438281722, 0.004309827907215549990447627635, 0.001963541004817085940087265941, -0.0008943234149703396950331270934, 6.105398965710550909211669004e-05, 6.18833202716769579476806784e-05  …  -8.193352719716318716845210401e-17, 4.678651669174662819840084076e-18, 6.290977208505213125174331796e-18, -2.036602398263703289116784981e-18, -5.514501237092254757475084934e-20, 2.007812537370097757991522041e-19, -4.634038429218728645259136977e-20, -6.570708157395391346804408734e-21, 5.946190599978746106321751204e-21, -9.218630047818848213294095374e-22], (0.5753999999999999999999999996, 0.8000000000000000000000000006)), var"#DTg#15"{Poly,BigFloat,Poly,Poly,Poly,Poly,Poly,Poly}(Poly(BigFloat[0.5753999999999999999999999996, 0.8000000000000000000000000006, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [32]:
for (k, a) in enumerate(gn.coeffs)
    println("$(k-1): $(a)")
end

0: -0.0001511713350611291656840259461
1: -1.256390384839309113454966115
2: 0.2468156475594798891903756139
3: 0.05671913449535499518131215337
4: -0.03593899510003976630438281692
5: 0.004309827907215549990447627761
6: 0.001963541004817085940087265859
7: -0.0008943234149703396950331270792
8: 6.105398965710550909211669497e-05
9: 6.188332027167695794768067623e-05
10: -2.032064217816506082080031603e-05
11: -5.917508310565041252403351028e-07
12: 1.993110264488937754875573495e-06
13: -4.460808064168219715751740601e-07
14: -7.091465500270745524324392239e-08
15: 5.999985897082526939375574855e-08
16: -9.224090264909730396057487991e-09
17: -3.081048454699410800420030238e-09
18: 1.652284575905043706735798251e-09
19: -1.530219765235557359682330951e-10
20: -1.096154081162629255138868031e-10
21: 4.22375356017612712026145581e-11
22: -1.004640487249960190548467289e-12
23: -3.558676617626400441095391722e-12
24: 1.004423856884389818876374579e-12
25: 6.516615444194818766762551315e-14
26: -1.079237625624183

In [33]:
a = gn(1)
alpha = 1/a

-1.690302971405244853343780156

In [34]:
@everywhere begin

# interval datatype
struct Interval
    lo::BigFloat
    hi::BigFloat
    Interval(lo, hi) = lo > hi ? error("bad Interval") : new(lo, hi)
    Interval(a) = Interval(a, a)
    Interval(i::Interval) = new(i.lo, i.hi)
end

# membership
function in(a::Number, x::Interval)
    x.lo <= a <= x.hi
end

# subset
function in(x::Interval, y::Interval)
    y.lo <= x.lo && x.hi <= y.hi
end

(==)(x::Interval, y::Interval) = x.lo == y.lo && x.hi == y.hi

(<)(x::Interval, y::Interval) = x.hi < y.lo
(>)(x::Interval, y::Interval) = y<x

(<)(x::BigFloat, y::Interval) = x < y.lo
(<)(x::Interval, y::BigFloat) = x.hi < y

(>)(x::BigFloat, y::Interval) = y<x
(>)(x::Interval, y::BigFloat) = y<x

zero(i::Interval) = Interval(0)
one(i::Interval) = Interval(1)

function abs(x::Interval)
    if 0 in x
        return Interval(0, max(abs(x.lo), abs(x.hi)))
    else
        return Interval(min(abs(x.lo), abs(x.hi)), max(abs(x.lo), abs(x.hi)))
    end
end

import Base: setrounding

function +(x::Interval, y::Interval)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = x.lo+y.lo
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = x.hi+y.hi
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

(-)(x::Interval) = Interval(-x.hi, -x.lo)
(-)(x::Interval, y::Interval) = x+(-y)

function +(x::Interval, a::Number)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = x.lo+a
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = x.hi+a
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

(+)(a::Number, x::Interval) = x+a
(-)(x::Interval, a::Number) = x+(-a)
(-)(a::Number, x::Interval) = -x+a

function *(x::Interval, y::Interval)
    setrounding(BigFloat, RoundDown)
    lo::BigFloat = min(x.lo*y.lo, x.lo*y.hi, x.hi*y.lo, x.hi*y.hi)
    setrounding(BigFloat, RoundUp)
    hi::BigFloat = max(x.lo*y.lo, x.lo*y.hi, x.hi*y.lo, x.hi*y.hi)
    setrounding(BigFloat, RoundNearest)
    Interval(lo, hi)
end

function *(x::Interval, a::Number)
    if a >= 0
        setrounding(BigFloat, RoundDown)
        c::BigFloat = x.lo*a
        setrounding(BigFloat, RoundUp)
        d::BigFloat = x.hi*a
        setrounding(BigFloat, RoundNearest)
        Interval(c, d)
    else
        setrounding(BigFloat, RoundDown)
        e::BigFloat = x.hi*a
        setrounding(BigFloat, RoundUp)
        f::BigFloat = x.lo*a
        setrounding(BigFloat, RoundNearest)
        Interval(e, f)
    end
end

(*)(a::Number, x::Interval) = x*a

function iRec(x::Interval)
    if 0 in x
        error("division by zero")
    else
        setrounding(BigFloat, RoundDown)
        lo::BigFloat = 1/x.hi
        setrounding(BigFloat, RoundUp)
        hi::BigFloat = 1/x.lo
        setrounding(BigFloat, RoundNearest)
        Interval(lo, hi)
    end
end

(/)(x::Interval, y::Interval) = x*iRec(y)
(/)(x::Interval, a::Number) = x*(1/a)
(/)(a::Number, x::Interval) = a*iRec(x)

function ^(x::Interval, n::Integer)
    if n%2 == 1 || x.lo >= 0
        # odd power
        setrounding(BigFloat, RoundDown)
        a::BigFloat = x.lo^n
        setrounding(BigFloat, RoundUp)
        b::BigFloat = x.hi^n
        setrounding(BigFloat, RoundNearest)
        return Interval(a, b)
    elseif x.hi < 0
        setrounding(BigFloat, RoundDown)
        c::BigFloat = x.hi^n
        setrounding(BigFloat, RoundUp)
        d::BigFloat = x.lo^n
        setrounding(BigFloat, RoundNearest)
        return Interval(c, d)
    else
        setrounding(BigFloat, RoundUp)
        e::BigFloat = x.lo^n
        f::BigFloat = x.hi^n
        setrounding(BigFloat, RoundNearest)
        return Interval(0, max(e, f))
    end
end

function sqrt(x::Interval)
    half = BigFloat(1)/BigFloat(2)
    @assert half*BigFloat(2) == BigFloat(1)
    @assert x.lo >= BigFloat(0)
    setrounding(BigFloat, RoundDown)
    a = x.lo^half
    setrounding(BigFloat, RoundUp)
    b = x.hi^half
    setrounding(BigFloat, RoundNearest)
    Interval(a, b)
end

comparable(x::Interval, y::Interval) = x.hi <= y.lo || x.lo >= y.hi

setrounding(t::Type{Float64}, mode) = nothing
#setrounding(t::Type{Double64}, mode) = nothing

function bound_quasi_power_careful(k0, g_norm::Interval)
    @assert(g_norm.hi < Interval(1))
    @assert(g_norm.hi > Interval(0))
    # important to make the interval trivial, here
    g_norm = Interval(g_norm.hi, g_norm.hi)
    k = k0 + 1
    latest = k * g_norm ^ (k - 1)
    greatest_upper = latest.hi
    greatest_lower = latest.lo
    #println(latest)
    
    # a new upper bound strictly below a previous lower bound
    # indicates that we have passed the maximum.
    # at that point, the greatest upper bound found so far
    # and the greatest lower bound found so far
    # together bound the maximum of the graph.
    
    while latest.hi >= greatest_lower
        if latest.lo > greatest_lower
            greatest_lower = latest.lo
        end
        if latest.hi > greatest_upper
            greatest_upper = latest.hi
        end
        k = k + 1
        latest = k * g_norm ^ (k - 1)
        #println(latest)
    end
    Interval(greatest_lower, greatest_upper)
end

# here, we would make the argument that
# the rounded version of the quasi power
# is not order reversing at any point.

# this means that it should be sufficient to
# compute upper bounds only.

function bound_quasi_power(k0, g_norm::Interval)
    @assert(g_norm.hi < Interval(1))
    @assert(g_norm.hi > Interval(0))
    # important to make the interval trivial, here
    k = k0 + 1
    setrounding(BigFloat, RoundUp)
    latest = k * g_norm.hi ^ (k - 1)
    greatest_upper = latest
    while latest >= greatest_upper
        if latest > greatest_upper
            greatest_upper = latest
        end
        k = k + 1
        latest = k * g_norm.hi ^ (k - 1)
    end
    setrounding(BigFloat, RoundNearest)
    Interval(0, greatest_upper)
end

# we established that the two methods produce the same results
#for k in 1:100
#    p = Int(floor(rand()*500))
#    x = 1-0.001*rand()
#    bound1 = bound_quasi_power(p, Interval(x)).hi
#    bound2 = bound_quasi_power_crude(p, Interval(x)).hi
#    if bound1 != bound2
#        println("$x: $bound1 != $bound2")
#    end
#end

function balanced(i::Interval)
    a = max(abs(i.lo), abs(i.hi))
    Interval(-a, a)
end

function nonnegative(i::Interval)
    a = max(i.lo, BigFloat(0))
    @assert i.hi>=0
    Interval(a, i.hi)
end

function hull(v::Array{Interval, 1})
    lo = min([i.lo for i in v]...)
    hi = max([i.hi for i in v]...)
    Interval(lo, hi)
end

unit_vector(t::Type, k::Integer) = [j == k ? t(1) : t(0) for j in 0:N]
    
end

In [35]:
@everywhere begin

struct Rectangle
    re::Interval
    im::Interval
    Rectangle(re::Interval, im::Interval) = new(re, im)
    Rectangle(re, im) = new(Interval(re), Interval(im))
    Rectangle(re) = new(Interval(re), Interval(0))
end

in(x::Rectangle, z::Rectangle) = x.re in z.re && x.im in z.im
zero(z::Rectangle) = Rectangle(0, 0)
one(z::Rectangle) = Rectangle(1, 0)

(-)(z::Rectangle) = Rectangle(-z.re, -z.im)

(+)(z::Rectangle, w::Rectangle) = Rectangle(z.re+w.re, z.im+w.im)
(-)(z::Rectangle, w::Rectangle) = Rectangle(z.re-w.re, z.im-w.im)

(*)(z::Rectangle, x::Interval) = Rectangle(z.re*x, z.im*x)
(/)(z::Rectangle, x::Interval) = z*(1/x)

function reciprocal(z::Rectangle)
    d = z.re^2 + z.im^2
    Rectangle(z.re/d, -z.im/d)
end

(/)(z::Rectangle, w::Rectangle) = z*reciprocal(w)

(*)(z::Rectangle, w::Rectangle) = Rectangle(z.re*w.re-z.im*w.im, z.re*w.im+z.im*w.re)

(+)(z::Rectangle, x::Interval) = Rectangle(z.re+x, z.im)
(-)(z::Rectangle, x::Interval) = Rectangle(z.re-x, z.im)

# derived
(*)(x::Interval, z::Rectangle) = z*x
(/)(x::Interval, z::Rectangle) = Rectangle(x)/z
(+)(x::Interval, z::Rectangle) = z+x
(-)(x::Interval, z::Rectangle) = -z+x

# we don't have sqrt of an interval yet.
abs(z::Rectangle) = sqrt(z.re^2 + z.im^2)

(^)(z::Rectangle, p::Integer) = russian(z, p)

Rectangle(z::Complex{BigFloat}) = Rectangle(z.re, z.im)
    
end

In [36]:
@everywhere begin

struct Trunc
    coeffs::Array{Interval, 1}
    Trunc(a::Array{Interval, 1}) = length(a) == 1+N ? new(a) : error("wrong #coeffs")
end

abs(t::Trunc) = sum(abs(a) for a in t.coeffs)

(^)(t::Trunc, p::Integer) = russian(t, p)

zero(t::Trunc) = Trunc([Interval(0) for k in 0:N])
one(t::Trunc) = Trunc([k == 0 ? Interval(1) : Interval(0) for k in 0:N])

# Vector space essentials
(+)(t::Trunc, u::Trunc) = Trunc(t.coeffs + u.coeffs)
(-)(t::Trunc) = Trunc(-t.coeffs)
(*)(t::Trunc, a::Union{Number, Interval}) = Trunc(t.coeffs * a)

# Vector space derived
(-)(t::Trunc, u::Trunc) = t+(-u)
(/)(t::Trunc, a::Interval) = t*(1/a)
(*)(a::Union{Number, Interval}, t::Trunc) = t*a

# I think that (*)(Array{T, 1}, Number) is defined already by default.
# But since Interval is not a subtype of Number,
# we must define (*)(Array{Interval, 1}, Interval).
(*)(a::Array{Interval, 1}, b::Interval) = [ak*b for ak in a]
(*)(b::Interval, a::Array{Interval, 1}) = a*b

# Convenience
function +(t::Trunc, a::Interval)
    coeffs = copy(t.coeffs)
    coeffs[1] += a
    Trunc(coeffs)
end

function -(t::Trunc, a::Interval)
    coeffs = copy(t.coeffs)
    coeffs[1] -= a
    Trunc(coeffs)
end

(+)(a::Interval, t::Trunc) = t+a
(-)(a::Interval, t::Trunc) = (-t)+a

# Algebra
function *(t::Trunc, u::Trunc)
    coeffs = fill(Interval(0), N+1)
    for j in 0:N
        for k in 0:N-j
            coeffs[1+j+k] += t.coeffs[1+j]*u.coeffs[1+k]
        end
    end
    Trunc(coeffs)
end

# Evaluation and composition of polynomial. Compact, but could be made more efficient?
#(t::Trunc)(x::Union{Interval, Rectangle, Trunc}) = sum(a*x^(k-1) for (k, a) in enumerate(t.coeffs))
function (t::Trunc)(x::Union{Interval, Rectangle, Trunc})
    total = t.coeffs[N+1]*x + t.coeffs[N]
    for k in N-1:-1:1
        total = total*x + t.coeffs[k]
    end
    total
end

function bound_high_order_product(t::Trunc, u::Trunc)
    coeffs = Dict{Int64, Interval}()
    for j in 0:N
        for k in 0:N
            if j+k > N
                a::Interval = t.coeffs[1+j]*u.coeffs[1+k]
                if haskey(coeffs, j+k)
                    coeffs[j+k] += a
                else
                    coeffs[j+k] = a
                end
            end
        end
    end
    sum(abs(a) for (k, a) in coeffs)
end

abs_star(t::Trunc) = sum(abs(t.coeffs[1+k]) for k in 1:N)

diff(t::Trunc) = Trunc([k == N ? Interval(0) : (1+k)*t.coeffs[2+k] for k in 0:N])

end

In [37]:
@everywhere begin

struct StdBall
    P::Trunc
    H::Interval
    G::Interval
    
    # we enforce the intervals h, g to be of the form [0, b].
    StdBall(p::Trunc, h::Number, g::Number) = new(p, Interval(0, h), Interval(0, g))
    StdBall(p::Trunc, h::Interval, g::Interval) = new(p, Interval(0, h.hi), Interval(0, g.hi))
end

zero(b::StdBall) = StdBall(zero(b.P), Interval(0), Interval(0))
one(b::StdBall) = StdBall(one(b.P), Interval(0), Interval(0))

# Abs
abs(b::StdBall) = nonnegative(abs(b.P) + b.H + balanced(b.G))

# Vector space operations

# Negative
(-)(b::StdBall) = StdBall(-b.P, b.H, b.G)

# Scalar multiples
(*)(b::StdBall, i::Interval) = StdBall(b.P*i, b.H*abs(i), b.G*abs(i))
(/)(b::StdBall, i::Interval) = b*(1/i)

# Add and subtract
(+)(b::StdBall, c::StdBall) = StdBall(b.P+c.P, b.H+c.H, b.G+c.G)
(-)(b::StdBall, c::StdBall) = b+(-c)

# Constant add and subtract
(+)(b::StdBall, i::Interval) = StdBall(b.P+i, b.H, b.G)
(-)(b::StdBall, i::Interval) = StdBall(b.P-i, b.H, b.G)

# Derived
(*)(i::Interval, b::StdBall) = b*i
(+)(i::Interval, b::StdBall) = b+i
(-)(i::Interval, b::StdBall) = (-b)+i

function *(b::StdBall, c::StdBall)
    PP_P = b.P*c.P
    PP_H = bound_high_order_product(b.P, c.P)
    PH = abs(b.P)*c.H
    PG = abs(b.P)*c.G

    HP = b.H*abs(c.P)
    HH = b.H*c.H
    HG = b.H*c.G
    
    GP = b.G*abs(c.P)
    GH = b.G*c.H
    GG = b.G*c.G
    
    P = PP_P
    H = PP_H + PH + HP + HH + GH + HG
    G = PG + GP + GG
    
    StdBall(P, H, G)
end

(^)(t::StdBall, p::Integer) = russian(t, p)

function abs_star(g::StdBall)
    x = abs_star(g.P) + Interval(0, g.G.hi) + Interval(-g.G.hi, g.G.hi)
    Interval(max(0, x.lo), x.hi)
end

# function ball compose

function (f::StdBall)(g::StdBall)
    if ~(f.H == f.G == Interval(0)) && ~(abs(g) in Interval(0, 1))
        error("composition not defined")
    end
    ball_1 = f.P(g)
    ball_3 = StdBall(zero(f.P), Interval(0), f.G)
    g_norm = abs(g)^(N+1)
    g_star_norm = abs_star(g)^(N+1)
    ball_2 = StdBall(zero(f.P), f.H*g_star_norm, f.H*(g_norm - g_star_norm))
    ball_1 + ball_2 + ball_3
end

function (f::StdBall)(g) #could be g::AbstractRigorousScalar
    if ~(f.H == f.G == Interval(0)) && ~(abs(g) in Interval(0, 1))
        error("composition not defined")
    end
    g_norm = abs(g)^(N + 1)
    r = f.H * g_norm + f.G
    f.P(g) + Interval(-r.hi, r.hi)
end

# could absorb this into earlier definition
#(t::Trunc)(x::StdBall) = sum(a*x^(k-1) for (k, a) in enumerate(t.coeffs))
function (t::Trunc)(x::StdBall)
    total = t.coeffs[N+1]*x + t.coeffs[N]
    for k in N-1:-1:1
        total = total*x + t.coeffs[k]
    end
    total
end

function diff_compose(f::StdBall, g::StdBall)
    if (f.H.hi > 0 || f.G.hi > 0) && ~(abs(g) in Interval(0, 1))
        error("diff composition not defined")
    end
    ball_1 = diff(f.P)(g)
    g_norm = abs(g)
    if f.G == f.H == Interval(0)
        norm_sup_H = norm_sup_G = 0
    else
        norm_sup_H = bound_quasi_power(N, g_norm)
        norm_sup_G = bound_quasi_power(0, g_norm)
    end
    # some sacrifice is made here! absorbing into general term!
    # an alternative is to absord into both high-order term and
    # the final polynomial coefficient.
    ball_2 = StdBall(zero(f.P), Interval(0), f.H * norm_sup_H)
    ball_3 = StdBall(zero(f.P), Interval(0), f.G * norm_sup_G)
    ball_1 + ball_2 + ball_3
end

diff_compose(f::StdBall, t::Trunc) = diff_compose(f, StdBall(t, 0, 0))

function basis_element(b::StdBall, k::Integer)
    if k <= N
        return StdBall(Trunc(unit_vector(Interval, k)), 0, 0)
    else
        return StdBall(Trunc(fill(Interval(0), N+1)), 1, 0)
    end
end
    
end

In [38]:
@everywhere begin

struct Domain
    c::Interval
    r::Interval
    Domain(c::Interval, r::Interval) = r.lo <= 0 ? error("domain radius") : new(c, r)
end

in(z::Union{BigFloat, Interval}, d::Domain) = abs(z-d.c) < d.r
in(z::Rectangle, d::Domain) = abs(z-Rectangle(d.c, 0)) < d.r
(==)(d::Domain, e::Domain) = d.c==e.c && d.r==e.r
    
end

In [39]:
@everywhere begin

struct Ball
    std::StdBall
    dom::Domain
end

# negative
(-)(f::Ball) = Ball(-f.std, f.dom)

# scalar multiply
(*)(f::Ball, i::Interval) = Ball(f.std*i, f.dom)
(*)(i::Interval, f::Ball) = f*i
(/)(f::Ball, i::Interval) = f*(1/i)

(*)(f::Ball, a::Number) = f*Interval(a)
(*)(a::Number, f::Ball) = f*a

# ball add
(+)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std+g.std, f.dom) : error("domain mismatch")
(-)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std-g.std, f.dom) : error("domain mismatch")

# scalar add
(+)(f::Ball, a::Interval) = Ball(f.std+a, f.dom)
(-)(f::Ball, a::Interval) = Ball(f.std-a, f.dom)

# ball mul
(*)(f::Ball, g::Ball) = f.dom == g.dom ? Ball(f.std*g.std, f.dom) : error("domain mismatich")

# ball pow
(^)(f::Ball, n::Integer) = Ball(f.std^n, f.dom)

# ball eval/compose
(f::Ball)(i) = f.std((i - f.dom.c)/f.dom.r)
(f::Ball)(g::Ball) = Ball(f.std((g.std - f.dom.c)/f.dom.r), g.dom)

# diff_compose
diff_compose(f::Ball, g::Ball) = Ball(diff_compose(f.std, (g.std - f.dom.c)/f.dom.r)/f.dom.r, g.dom)

# allow differentiation for composition
function diff(f::Ball)
    df(g::Ball) = diff_compose(f, g)
end

# identity with respect to domain
function identity(f::Ball)
    coeffs = fill(Interval(0), N+1)
    coeffs[1] = f.dom.c
    coeffs[2] = f.dom.r
    Ball(StdBall(Trunc(coeffs), Interval(0), Interval(0)), f.dom)
end

basis_element(f::Ball, k::Integer) = Ball(basis_element(f.std, k), f.dom)

function Ball(p::Poly, h::Number, g::Number)
    c, r = p.cr
    dom = Domain(Interval(c), Interval(r))
    t = Trunc(map(Interval, p.coeffs))
    s = StdBall(t, h, g)
    b = Ball(s, dom)
end

abs(b::Ball) = abs(b.std)

function (m::Array{Interval, 2})(b::Ball, coeff_HH::Interval)
    PP = m*b.std.P.coeffs
    # assume all of G is in the polynomial part
    PG = [hull(m[1+k,:])*balanced(b.std.G) for k in 0:N]
    P = Trunc(PP + PG)
    H = (b.std.H + b.std.G)*abs(coeff_HH)
    G = Interval(0)
    Ball(StdBall(P, H, G), b.dom)
end
    
end

In [40]:
@everywhere dom = Domain(Interval(c), Interval(r))

In [41]:
@everywhere I = eye(BigFloat, N+1);

In [42]:
@everywhere Lambda_pp = map(Interval, inv(J-I))

In [43]:
@everywhere begin

function gauss(A_::Matrix)
    A = copy(A_)      
    m, n = size(A)
    @assert m <= n

    for k in 1:m
        
        # seek largest nonzero A[j,k] across j
        best, abs_best = k, abs(A[k,k])
        for i in k+1:m
            if abs(A[i,k]) > abs_best
                best, abs_best = i, abs(A[i,k])
            end
        end

        # swap rows to position pivot
        A[k,:], A[best,:] = A[best,:], A[k,:]

        # divide row by pivot
        pivot = A[k,k]
        for j in k:n
            A[k,j] /= pivot
        end

        # clear zeros above and below
        for k_that in 1:m
            if k_that != k
                ratio = A[k_that,k]/A[k,k]
                for j in k:n
                    A[k_that,j] -= ratio*A[k,j]
                end
            end
        end
    end
    A
end

#invert(A::Matrix) = gauss(hcat(A, eye(typeof(A[1,1]), size(A)[1])))[:,size(A)[2]+1:end]
    
end

In [44]:
# success here means that Lambda_pp, and hence Lambda itself,
# is invertible.
#@everywhere Lambda_pp_inv = invert(Lambda_pp);

In [45]:
@everywhere function Phi(b0::Ball, Lambda_pp::Array{Interval, 2})
    Tb0, DTb0 = T_and_DT(b0)
    Phib0 = b0 - Lambda_pp(Tb0-b0, Interval(1))
end

In [46]:
B0 = Ball(gn, 0, 0);

In [47]:
@time Phi_B0 = Phi(B0, Lambda_pp);

  0.832063 seconds (12.68 M allocations: 578.383 MiB, 5.52% gc time)


In [48]:
epsilon = abs(Phi_B0-B0)

Interval(0.0, 5.59105108687788961834971942e-22)

In [49]:
rho_log10 = Int(round(0.5+log10(epsilon.hi)))
rho = BigFloat(10)^rho_log10

9.999999999999999999999999995e-22

In [50]:
B1 = Ball(gn, 0, rho);

In [51]:
open(f -> serialize(f, B0), "$(prefix)/b0.jls", "w")
open(f -> serialize(f, B1), "$(prefix)/b1.jls", "w")

In [52]:
@everywhere begin
    
B0 = open(deserialize, "$(prefix)/b0.jls")
B1 = open(deserialize, "$(prefix)/b1.jls")
rho = B1.std.G.hi
    
end

In [53]:
@everywhere psi(x) = (x-B1.dom.c)/B1.dom.r
a = B1(Interval(1))

Interval(-0.5916099166344381501406243584, -0.5916099166344381501386243439)

In [54]:
@everywhere begin
T_B1, DT_B1 = T_and_DT(B1)
end

# do we really need this everywhere?
# can we serialise functions?

In [55]:
dB_H = basis_element(B1, N+1)
DPhi_B1_dB_H = -Lambda_pp(DT_B1(dB_H), Interval(1))
norm_H = abs(DPhi_B1_dB_H)

Interval(0.0, 0.005313973044591245551281963271)

In [56]:
@everywhere E = [basis_element(B1, k) for k in 0:N+1];

In [57]:
# Precompute DT(B^1)(E_k) for all basis vectors, E_k.
@time DT_B1_E = pmap(DT_B1, E);

  1.578745 seconds (1.68 M allocations: 82.183 MiB, 0.39% gc time)


In [58]:
open(f -> serialize(f, DT_B1_E), "$(prefix)/DT_B1_E.jls", "w")

In [59]:
@everywhere begin

DT_B1_E = open(deserialize, "$(prefix)/DT_B1_E.jls")
    
function bound_kappa_naively(k::Int)
    DPhi_B1_E_k = E[k] - Lambda_pp(DT_B1_E[k]-E[k], Interval(1))
    norm = abs(DPhi_B1_E_k)
end

end

In [60]:
@time norms = pmap(bound_kappa_naively, [k for k in 1:N+2])

  0.558855 seconds (1.10 M allocations: 56.908 MiB, 2.80% gc time)


43-element Array{Interval,1}:
 Interval(0.0, 1.626908986389681940967032847e-17)
 Interval(0.0, 8.196309030981745008428412323e-18)
 Interval(0.0, 5.651604190718895482051113429e-18)
 Interval(0.0, 4.473176799306854584609027429e-18)
 Interval(0.0, 3.12920987978943308651098472e-18) 
 Interval(0.0, 2.777990646511471032861495017e-18)
 Interval(0.0, 2.163828791913701591995265205e-18)
 Interval(0.0, 1.853818798049203130482125607e-18)
 Interval(0.0, 1.505330818006822409063382978e-18)
 Interval(0.0, 1.267527313579883692133994731e-18)
 Interval(0.0, 1.042718003095593825823300606e-18)
 Interval(0.0, 8.698763882143882948013176092e-19)
 Interval(0.0, 7.213001327185265993752275943e-19)
 ⋮                                               
 Interval(0.0, 0.0005954004123502826182270213516)
 Interval(0.0, 0.0007268477294049414569309186962)
 Interval(0.0, 0.0007888229226959855037313110136)
 Interval(0.0, 0.0007741671739594375579590396652)
 Interval(0.0, 0.000701362977229968034199187984) 
 Interval(0.0, 0.000

In [61]:
norms[end] = norm_H
kappa = hull(norms).hi

0.005313973044591245551281963271

In [62]:
Phi_is_contractive = kappa < BigFloat(1)

true

In [63]:
fixed_point_exists = epsilon < rho*(Interval(1)-kappa)

true

In [64]:
println("Parameters:")
println("Power                     = $d")
println("Domain centre             = $c")
println("Domain radius             = $r")
println("Term G^u_0 (approx)       = $(gn.coeffs[1])")
println("Degree (reduced)          = $N")
println("Degree (original)         = $(N*d)")
println("Floating point type       = $(BigFloat)")
println("Precision (digits approx) = $P")
println("Precision (bits)          = $(precision(BigFloat))")
println()
println("Bounds:")
println("epsilon       = $(epsilon.hi)")
println("rho           = $rho")
#println("composition1  = $(norm1.hi)")
#println("composition2  = $(norm2.hi)")
println("kappa         = $kappa.hi")
println("rho*(1-kappa) = $((rho*(Interval(1)-kappa)).lo)")
println()
println("Conclusions:")
#println("proved domain extension   = $domain_extension")
println("proved phi is contractive = $Phi_is_contractive")
println("proved fixed point exists = $fixed_point_exists")
if fixed_point_exists
    println()
    println("Constants:")
    println("a = g(1)       = $(B1(1))")
    println("alpha = 1/g(1) = $(1/B1(1))")
end
println("Processors: 1+$(length(procs))")

Parameters:
Power                     = 4
Domain centre             = 0.5753999999999999999999999996
Domain radius             = 0.8000000000000000000000000006
Term G^u_0 (approx)       = -0.0001511713350611291656840259461
Degree (reduced)          = 41
Degree (original)         = 164
Floating point type       = BigFloat
Precision (digits approx) = 27
Precision (bits)          = 89

Bounds:
epsilon       = 5.59105108687788961834971942e-22
rho           = 9.999999999999999999999999995e-22
kappa         = 0.005313973044591245551281963271.hi
rho*(1-kappa) = 9.946860269554087544487180361e-22

Conclusions:
proved phi is contractive = true
proved fixed point exists = true

Constants:
a = g(1)       = Interval(-0.5916099166344381501406243584, -0.5916099166344381501386243439)
alpha = 1/g(1) = Interval(-1.690302971405244853346637306, -1.690302971405244853340923013)
Processors: 1+6


In [65]:
open(f -> serialize(f, [epsilon, rho, kappa]), "$(prefix)/erk.jls", "w")

In [66]:
# now we must decide what to do with the high-order
# and general bounds we could take each general
# bound [0, g] and add [-g, g] to every polynomial
# term for the high-order bound [0, h] we form
# [-h, h]+[-g, g].  in this way we form what i call
# the augmented matrix $M$

M_aug = [Interval(0) for j in 1:N+2, k in 1:N+2]

for k in 1:N+2
    b_H = balanced(DT_B1_E[k].std.H)
    b_G = balanced(DT_B1_E[k].std.G)
    for j in 1:N+1
        M_aug[j,k] = DT_B1_E[k].std.P.coeffs[j] + b_G
    end
    M_aug[N+2,k] = b_H + b_G
end
M_aug;

# ignoring the high-order parts yields the so-called PP matrix
M_PP = M_aug[1:N+1,1:N+1];

In [67]:
# we now find approximate eigvenvalues and vectors of the PP matrix

In [68]:
using GenericSchur

M_approx = map(i -> i.lo, M_PP)
@time vals_rev, vecs_rev = GenericSchur.eigen(M_approx);

  3.573160 seconds (16.19 M allocations: 756.888 MiB, 6.90% gc time)


In [69]:
vals = reverse(vals_rev)

42-element Array{Complex{BigFloat},1}:
      8.163158323607492931540447915 + 0.0im
        7.2846862170733433641227755 + 0.0im
     0.2918384089835451045914907148 + 0.0im
     0.1225016054274047603087610506 + 0.0im
    0.04302851883343190636179024568 + 0.0im
     0.0150066433322915697807655657 + 0.0im
   0.005251575462679995686199226692 + 0.0im
   0.001838337900281377069343431905 + 0.0im
  0.0006434270306182293151403213955 + 0.0im
  0.0002251993440550281914811026444 + 0.0im
  7.882025946035435558925834189e-05 + 0.0im
  2.758728107153568481030702521e-05 + 0.0im
  9.655611709530266331564946238e-06 + 0.0im
                                    ⋮      
 -5.712158191727309113453333614e-06 + 0.0im
 -1.632055319693357222145224486e-05 + 0.0im
 -4.663238392763636112523767256e-05 + 0.0im
 -0.0001332554049810915450138268183 + 0.0im
 -0.0003808771870070765542643373881 + 0.0im
  -0.001089193634100644224408130718 + 0.0im
  -0.003117652548854664023123872338 + 0.0im
   -0.00893787441231092859131038867 +

In [70]:
open(f -> serialize(f, vals), "$(prefix)/val.jls", "w")

In [71]:
vecs_rev_re = map(x->BigFloat(x), vecs_rev)
vecs = vecs_rev_re[1:N+1,N+1:-1:1];

In [72]:
normalised_T = [vecs[:,k]/sum(map(abs, vecs[:,k])) for k in 1:N+1]
vecs = [normalised_T[k][j] for j in 1:N+1, k in 1:N+1];

In [73]:
eigen_error = sum(map(abs, M_PP*vecs[:,1] - vals[1]*vecs[:,1]))

Interval(0.0, 1.627580703272574871472850542e-17)

In [74]:
@everywhere begin

span(i::Interval) = Interval(0, (i-i).hi)

phi(p::Poly) = p.coeffs[1]
phi(b::Ball) = b.std.P.coeffs[1] + balanced(b.std.G)
    
end

In [75]:
phi(B0)

Interval(-0.0001511713350611291656840259461, -0.0001511713350611291656840259461)

In [76]:
span(phi(B0))

Interval(0.0, 0.0)

In [77]:
phi(B1)

Interval(-0.0001511713350611291666840259463, -0.0001511713350611291646840259459)

In [78]:
span(phi(B1))

Interval(0.0, 2.000000000428778400695533731e-21)

In [79]:
i = 2
val_approx = vals[i]
vec_approx = vecs[1:end,i]

# normalize properly
f0_poly_coeffs = vec_approx/vec_approx[1]*val_approx.re
f0_poly_coeffs[1]

7.2846862170733433641227755

In [80]:
f0_poly = Poly(f0_poly_coeffs, (c, r))

Poly(BigFloat[7.2846862170733433641227755, 7.166839843174802182245915569, -5.433001676297059973436373972, -0.8162303438668710533363711057, 1.274860776188583447150440838, -0.2771613530843119827930050898, -0.08203153501419121655577890955, 0.05969385630085114918568681662, -0.00796816236743723113274201797, -0.004473236961734717358033556199  …  2.600848274698282845798084057e-14, -2.762978368693800165704259949e-15, -1.799360683171407950012539687e-15, 7.412929944938900311682667165e-16, -2.105549282101549703037707637e-17, -6.688591381091639753139283659e-17, 1.906155549151203684505094714e-17, 9.009344499839874884290228882e-19, -2.647183245119983781503755563e-18, 4.44499369252030181177108078e-20], (0.5753999999999999999999999996, 0.8000000000000000000000000006))

In [81]:
abs(DTgn(f0_poly)-f0_poly*vals[i].re)

1.347793196924461281025465804e-16

In [82]:
@everywhere begin
    
function newtonstep2(p::Poly)
    Fp = DTgn(p) - p.coeffs[1]*p
    Z = [BigFloat(0) for j in 1:N+1,k in 1:N+1]
    for j in 1:N+1
        for k in 1:N+1
            if k == 1
                Z[j, k] += p.coeffs[j]
            end
            if j == k
                Z[j, k] += p.coeffs[1]
            end
        end
    end
    q = Poly(p.coeffs - inv(J-Z)*Fp.coeffs, p.cr)
    q, abs(Fp)
end

function newton2(p0::Poly, n::Integer)
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1, error = newtonstep2(p0)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end
    
end

In [83]:
@time fn = newton2(f0_poly, 20);

1: 1.347793196924461281025465804e-16
2: 5.714223028854146813845124733e-25
  0.149769 seconds (2.50 M allocations: 116.809 MiB, 10.64% gc time)


In [84]:
# memory was exceeded with 5 workers, one of which died.
# attempting again with 4.
# we could look at reassigning DT_B1_E perhaps, if we
# are sure that it is no longer needed?
# it is needed when bounding kappa_hat.
# it could be removed and then reloaded?
# for now, just trying 4 procs instead.
# we could also force gc() garbage collection, although
# I would guess that this is forced already when
# memory becomes a problem.
workers()

6-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7

In [85]:
@time Delta_PP = matrix_elements(DTgn, poly_basis);

  0.101603 seconds (106.16 k allocations: 3.575 MiB)


In [86]:
Delta_mod = [BigFloat(0) for j in 1:N+1,k in 1:N+1]
for j in 1:N+1
    for k in 1:N+1
        Delta_mod[j,k] = Delta_PP[j,k]
        if k == 1
            Delta_mod[j,k] = Delta_mod[j,k] - fn.coeffs[j]
        end
        if j == k
            Delta_mod[j,k] = Delta_mod[j,k] - fn.coeffs[1]
        end
    end
end
Delta_mod;

In [87]:
L_pp = map(Interval, inv(Delta_mod)); #invert

# now we check that the resulting interval matrix is invertible
#@time invert(L_pp);

In [88]:
V0 = Ball(StdBall(Trunc(map(Interval, fn.coeffs)), BigFloat(0), BigFloat(0)), dom);

In [89]:
eigenfunction_error = abs(V0*phi(V0) - DT_B1(V0))

Interval(0.0, 1.281185478249539988124623527e-17)

In [90]:
V0.dom == B1.dom

true

In [91]:
open(f -> serialize(f, L_pp), "$(prefix)/L_pp.jls", "w")
open(f -> serialize(f, V0), "$(prefix)/V0.jls", "w")

In [92]:
@everywhere begin

L_pp = open(deserialize, "$(prefix)/L_pp.jls")
V0 = open(deserialize, "$(prefix)/V0.jls")
    
function Psi(v::Ball, L_pp::Array{Interval, 2})
    Psi_v = v - L_pp(DT_B1(v)-v*phi(v), -Interval(1)/V0.std.P.coeffs[1])
end
    
end

In [93]:
@time epsilon_hat = abs(Psi(V0, L_pp)-V0)

  0.128084 seconds (5.44 M allocations: 244.456 MiB, 13.18% gc time)


Interval(0.0, 3.21702283848831572380732293e-17)

In [94]:
rho_hat = BigFloat(10)^Int(round(0.5+log10(epsilon_hat.hi)))

1.0e-16

In [95]:
V1 = Ball(fn, 0, rho_hat);

In [96]:
open(f -> serialize(f, V1), "$(prefix)/V1.jls", "w")

In [97]:
println(phi(V1).lo)
println(phi(V1).hi)
println(1+rho_hat)

7.28468621707334326430893103
7.284686217073343464308931042
1.0000000000000001


In [98]:
rho_hat

1.0e-16

In [99]:
@everywhere begin
    
V1 = open(deserialize, "$(prefix)/V1.jls")

function DPsi(b::Ball, L_pp::Array{Interval, 2})
    function DPsi_b(db::Ball)
        DF = DTb1(db)-phi(b)*db-phi(db)*b
        DPsi_b_db = db - L_pp(DF, -Interval(1)/phi(V0))
    end
    DPsi_b
end

function bound_kappa_hat_naively(k::Int)
    b = V1
    DPsi_b_es_k = E[k] - L_pp(DT_B1_E[k]-phi(b)*E[k]-phi(E[k])*b, -Interval(1)/phi(V0))
    norm = abs(DPsi_b_es_k)
end
    
end

In [100]:
EH = basis_element(V1, N+1)
DT_G_EH = DT_B1(EH)
DPsi_V1_EH = (Interval(1)-phi(V1)/phi(V0))*EH -L_pp(DT_G_EH, -Interval(1)/phi(V0))
norm_H = abs(DPsi_V1_EH)

Interval(0.0, 0.0008995071552257036770101044016)

In [101]:
@time norms_v = pmap(bound_kappa_hat_naively, [k for k in 1:N+2])

  0.141453 seconds (267.38 k allocations: 13.249 MiB)


43-element Array{Interval,1}:
 Interval(0.0, 9.329627616842611271349406694e-16)
 Interval(0.0, 1.393150148818237198095473424e-16)
 Interval(0.0, 1.010221886406813822014145492e-16)
 Interval(0.0, 9.588341133353346494105008278e-17)
 Interval(0.0, 7.264125048053308143912493088e-17)
 Interval(0.0, 5.614306796449927714779966259e-17)
 Interval(0.0, 4.45097190979307974957069243e-17) 
 Interval(0.0, 3.640312067115662242256840506e-17)
 Interval(0.0, 3.026070669111788726325900055e-17)
 Interval(0.0, 2.585806739382328688335463369e-17)
 Interval(0.0, 2.257274026423953202047048177e-17)
 Interval(0.0, 2.022875480175057984253351158e-17)
 Interval(0.0, 1.84846916550360645685966686e-17) 
 ⋮                                               
 Interval(0.0, 8.173315838298215791478034227e-05)
 Interval(0.0, 9.977749318858870361031168186e-05)
 Interval(0.0, 0.0001082850927535213505783327019)
 Interval(0.0, 0.0001062732355094579301040935738)
 Interval(0.0, 9.627909237686340644909331774e-05)
 Interval(0.0, 8.237

In [102]:
norms_v[end] = norm_H
kappa_hat = hull(norms_v)

Interval(0.0, 0.0008995071552257036770101044016)

In [103]:
epsilon_hat < rho_hat*(Interval(1)-kappa_hat)

true

In [104]:
epsilon_hat

Interval(0.0, 3.21702283848831572380732293e-17)

In [105]:
rho_hat*(Interval(1)-kappa_hat)

Interval(9.991004928447742963229898936e-17, 1.0e-16)

In [106]:
println(phi(V1).lo)
println(phi(V1).hi)
println(1+rho_hat)

7.28468621707334326430893103
7.284686217073343464308931042
1.0000000000000001


# Critical scaling of additive noise

In [107]:
@everywhere function make_noise_functions(G)
    
    a = G(1)
    
    X = identity(G)
    XQa = X*Q(a)
    GXQa = G(XQa)
    QGXQa = Q(GXQa)
    GQGXQa = G(QGXQa)
    
    Gdash = diff(G)
    GdashQa = Gdash(XQa)
    GdashQGXQa = Gdash(QGXQa)
    
    t2 = (1/a^2)
    t3 = ((1/a)*GdashQGXQa*Qdash(GXQa))^2
    
    function T_noise(H)
        term2 = t2*H(QGXQa)
        term3 = t3*H(XQa)
        lhs = term2 + term3
        return lhs
    end

    function DT_noise(H)
        function DT_noise_H(K)
            term2 = t2*K(QGXQa)
            term3 = t3*K(XQa)
            lhs = term2 + term3
            return lhs
        end
        return DT_noise_H
    end

    function F_noise(H)
        lhs = T_noise(H)
        rhs = phi(H)^2*H
        return lhs - rhs
    end
    
    function DF_noise(H)
        function DF_noise_H(K)
            term2 = t2*K(QGXQa)
            term3 = t3*K(XQa)
            lhs = term2 + term3
            rhs = 2*phi(H)*phi(K)*H + phi(H)^2*K
            return lhs - rhs
        end
        return DF_noise_H
    end
    
    return T_noise, DT_noise, F_noise, DF_noise
end

In [108]:
T_noise, DT_noise, F_noise, DF_noise = make_noise_functions(gn)

(T_noise, DT_noise, F_noise, DF_noise)

In [109]:
function newtonstep3(p::Poly)
    F_noise_p = F_noise(p)
    
    DF_noise_p = DF_noise(p)
    Gamma_noise_inv = matrix_elements(DF_noise_p, poly_basis);
    
    q = Poly(p.coeffs - inv(Gamma_noise_inv)*F_noise_p.coeffs, p.cr)
    q, abs(F_noise_p)
end

function newton3(p0::Poly, n::Integer)
    basis = [basis_element(p0, k) for k in 0:N]
    prev_error = BigFloat(10)^10
    for k in 1:n
        p1, error = newtonstep3(p0)
        if error < prev_error
            println("$k: $error")
            p0 = p1
            prev_error = error
        else
            break
        end
    end
    p0
end

newton3 (generic function with 1 method)

In [110]:
v_noise_coeffs = BigFloat[8.2439108542525868183984625308166, -6.0314979598335030543896628404458, -0.99906557440532477623573316007567, 2.6365397214358035500680010442941, -0.81393255099831777701118943916041, -0.2916501997294583937683387447899, 0.27455204544220932923048986000544, -0.04629827836891161318089224352731, -0.03003398317265670879893203510508, 0.017474018792551148952490914683149]

10-element Array{BigFloat,1}:
  8.243910854252586517532108701  
 -6.03149795983350323069771548   
 -0.9990655744053247300229259054 
  2.636539721435803507176842686  
 -0.8139325509983177386885699889 
 -0.291650199729458381447955162  
  0.2745520454422093470370214163 
 -0.04629827836891161479959677649
 -0.03003398317265670858233761464
  0.01747401879255115059175196279

In [111]:
v_noise_0 = Poly([k <= length(v_noise_coeffs) ? v_noise_coeffs[k] : BigFloat(0) for k in 1:N+1], (c, r))
v_noise_0(0), phi(v_noise_0)

(10.96210056734141160035635622, 8.243910854252586517532108701)

## Clear some space?

In [112]:
# aim to clear some variables
@everywhere begin
    DT_B1_E = 0
    # others?
end

In [113]:
v_noise_n = newton3(v_noise_0, 20);
# in future, save after each 5 steps, say!
# or quit when we don't improve by an order of magnitude?

1: 0.4548378523956301411684889173
2: 3.303755231084358012973987573e-17
3: 9.374555883625503769882070639e-25


In [114]:
open(f -> serialize(f, v_noise_n), "$(prefix)/noise_n.jls", "w")

@everywhere v_noise_n = deserialize("$(prefix)/noise_n.jls")

In [115]:
phi(v_noise_n)

8.243910854252586818398464812

In [116]:
@time Lambda_PP_inv_noise = matrix_elements(DF_noise(v_noise_n), poly_basis);

  0.076739 seconds (80.51 k allocations: 3.020 MiB)


In [117]:
@time Lambda_PP_noise = map(Interval, inv(Lambda_PP_inv_noise));

  0.009492 seconds (393.72 k allocations: 18.052 MiB)


In [118]:
open(f -> serialize(f, Lambda_PP_noise), "$(prefix)/Lambda_PP_noise.jls", "w")

@everywhere begin
Lambda_PP_noise = deserialize("$(prefix)/Lambda_PP_noise.jls");
end

In [119]:
#@time Lambda_PP_noise_inv_check = inv(Lambda_PP_noise);

In [120]:
@everywhere begin
    
T_noise, DT_noise, F_noise, DF_noise = make_noise_functions(B1)

function Phi_noise(b::Ball)
    b - Lambda_PP_noise(F_noise(b), -1/phi(b)^2)
end
    
end

# possible memory issue.
# swap is full.
# are these needed everywhere?

In [121]:
B0_noise = Ball(v_noise_n, BigFloat(0), BigFloat(0));

In [122]:
@time epsilon_noise = abs(Phi_noise(B0_noise) - B0_noise)

  0.127263 seconds (5.25 M allocations: 235.661 MiB, 17.69% gc time)


Interval(0.0, 3.095039612738337392230063913e-17)

In [123]:
rho_noise = Interval(BigFloat(10)^Int(round(0.5+log10(epsilon_noise.hi))))

Interval(1.0e-16, 1.0e-16)

In [124]:
B1_noise = Ball(v_noise_n, BigFloat(0), BigFloat(rho_noise.hi));

In [125]:
println(phi(B1_noise).lo)
println(phi(B1_noise).hi)
println(1+rho_noise.hi)

8.243910854252586718398464806
8.243910854252586918398464818
1.0000000000000001


In [126]:
open(f -> serialize(f, B0_noise), "$(prefix)/B0_noise.jls", "w")
open(f -> serialize(f, B1_noise), "$(prefix)/B1_noise.jls", "w")

@everywhere begin
B0_noise = deserialize("$(prefix)/B0_noise.jls");
B1_noise = deserialize("$(prefix)/B1_noise.jls");
end

In [127]:
@everywhere begin
    
function DPhi_noise(b::Ball, L_pp::Array{Interval, 2})
    DF_noise_b = DF_noise(b)
    function DPhi_noise_b(db::Ball)
        DF = DF_noise_b(db)
        DPhi_b_db = db - L_pp(DF, -Interval(1)/phi(V0)^2)
    end
    DPhi_noise_b
end

function bound_kappa_noise_naively(k::Int)
    b = B1_noise
    L_pp = Lambda_PP_noise
    DPhi_noise_b = DPhi_noise(b, L_pp)
    DPhi_noise_b_es_k = DPhi_noise_b(E[k])
    norm = abs(DPhi_noise_b_es_k)
end
    
end

In [128]:
EH = basis_element(B1_noise, N+1)
DT_noise_B1_EH = DT_noise(B1_noise)(EH)
DPhi_noise_B1_EH = (Interval(1)-(phi(B1_noise)/phi(B0_noise))^2)*EH -L_pp(DT_noise_B1_EH, -Interval(1)/phi(V0))
norm_H = abs(DPhi_noise_B1_EH)

Interval(0.0, 0.006028566358224530196083150527)

In [129]:
@everywhere V0 = open(deserialize, "$(prefix)/V0.jls")

# note: earlier, when we deserialize B1_noise,
# this ball is centered on V0, we should therefore
# deserialize V0 everywhere, along with it.

In [130]:
workers()

6-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7

In [131]:
@time norms_w = pmap(bound_kappa_noise_naively, [k for k in 1:N+2])

  1.354871 seconds (294.36 k allocations: 14.675 MiB)


43-element Array{Interval,1}:
 Interval(0.0, 1.15859757832023100624435421e-15) 
 Interval(0.0, 4.471804088298597881306967538e-17)
 Interval(0.0, 4.393632219002362797795737001e-17)
 Interval(0.0, 3.789735746098236639982995159e-17)
 Interval(0.0, 3.490390454639776185993090534e-17)
 Interval(0.0, 3.311965598858664097730194476e-17)
 Interval(0.0, 3.209198702235537271421384271e-17)
 Interval(0.0, 3.028655331274973925726992148e-17)
 Interval(0.0, 2.901650858372509061289572271e-17)
 Interval(0.0, 2.781279623371308867185799518e-17)
 Interval(0.0, 2.690956036742669082084165634e-17)
 Interval(0.0, 2.616857539862772686073713412e-17)
 Interval(0.0, 2.571397091395860120808045713e-17)
 ⋮                                               
 Interval(0.0, 1.896496243770233215797706578e-05)
 Interval(0.0, 2.315188165839067141747885728e-05)
 Interval(0.0, 2.51259434636099225986035696e-05) 
 Interval(0.0, 2.46591219456544364471888207e-05) 
 Interval(0.0, 2.23401298394356523875040804e-05) 
 Interval(0.0, 1.911

In [132]:
norms_w[end] = norm_H
kappa_noise = Interval(hull(norms_w).hi)

Interval(0.006028566358224530196083150527, 0.006028566358224530196083150527)

In [133]:
noise_existence = epsilon_noise < rho_noise*(Interval(1)-kappa_noise)

true

In [134]:
open(f -> serialize(f, norms_w), "$(prefix)/norms_w.jls", "w")

In [135]:
open(f -> serialize(f, norms_v), "$(prefix)/norms_v.jls", "w")

In [136]:
open(f -> serialize(f, norms), "$(prefix)/norms_g.jls", "w")

In [137]:
println("Degree of map at critical point, d:\n$(d)")
println("Domain centre, c:\n$(c)")
println("Domain radius, r:\n$(r)")
println("Truncation degree, N:\n$(N)")
println("Precision (digits approx.), P:\n$(P)")

println("\nepsilon:\n$(epsilon.hi)")
println("rho:\n$(rho)")
println("kappa:\n$(kappa)")

println("\nepsilon_hat:\n$(epsilon_hat.hi)")
println("rho_hat:\n$(rho_hat)")
println("kappa_hat:\n$(kappa_hat.hi)")

println("\nepsilon_noise:\n$(epsilon_noise.hi)")
println("rho_noise:\n$(rho_noise.hi)")
println("kappa_noise:\n$(kappa_noise.hi)")

println("\na=G(1) lower bound:\n$(B1(1).lo)")
println("a=G(1) upper bound:\n$(B1(1).hi)")
println("1+rho:\n$(1+rho)")

println("\nalpha=1/G(1) lower bound:\n$((1/B1(1)).lo)")
println("alpha=1/G(1) upper bound:\n$((1/B1(1)).hi)")
println("1+rho:\n$(1+rho)")

println("\ndelta=phi(V) lower bound:\n$(phi(V1).lo)")
println("delta=phi(V) upper bound:\n$(phi(V1).hi)")
println("1+rho_hat:\n$(1+rho_hat)")

println("\ngamma=phi(K) lower bound:\n$(phi(B1_noise).lo)")
println("gamma=phi(K) upper bound:\n$(phi(B1_noise).hi)")
println("1+rho_noise:\n$(1+rho_noise.hi)")

Degree of map at critical point, d:
4
Domain centre, c:
0.5753999999999999999999999996
Domain radius, r:
0.8000000000000000000000000006
Truncation degree, N:
41
Precision (digits approx.), P:
27

epsilon:
5.59105108687788961834971942e-22
rho:
9.999999999999999999999999995e-22
kappa:
0.005313973044591245551281963271

epsilon_hat:
3.21702283848831572380732293e-17
rho_hat:
1.0e-16
kappa_hat:
0.0008995071552257036770101044016

epsilon_noise:
3.095039612738337392230063913e-17
rho_noise:
1.0e-16
kappa_noise:
0.006028566358224530196083150527

a=G(1) lower bound:
-0.5916099166344381501406243584
a=G(1) upper bound:
-0.5916099166344381501386243439
1+rho:
1.000000000000000000001

alpha=1/G(1) lower bound:
-1.690302971405244853346637306
alpha=1/G(1) upper bound:
-1.690302971405244853340923013
1+rho:
1.000000000000000000001

delta=phi(V) lower bound:
7.28468621707334326430893103
delta=phi(V) upper bound:
7.284686217073343464308931042
1+rho_hat:
1.0000000000000001

gamma=phi(K) lower bound:
8.243910

In [139]:
rmprocs(procs)

Task (done) @0x00007f35a603abf0